In [25]:
import glob
import os
import pandas as pd
from os import listdir
from shutil import rmtree
from subprocess import check_output
import argparse
import subprocess
from multiprocessing import Pool


# Betaseries for BBx


In [119]:
# set variable paths 

ses_id='ses-1'
study_path='/projects/niblab/experiments/bbx'
input_dir = '/projects/niblab/experiments/bbx/data'    
deriv_dir= os.path.join(input_dir, 'preprocessed')
design_file='/projects/niblab/experiments/bbx/data/code/beta_design.fsf'
ev_path='/projects/niblab/experiments/bbx/data/onsets/trials'
preprocess_folder= os.path.join(input_dir, 'preprocessed')

subject_folders=sorted(glob.glob('/projects/niblab/experiments/bbx/data/preprocessed/sub-*'))
# get subject ids
subject_ids=[x.split('/')[-2] for x in glob.glob('/projects/niblab/experiments/bbx/data/preprocessed/sub-*/ses-1')]



data_dict={}

In [120]:

def chunks(l,n):
    return [l[i:i+n] for i in range(0, len(l), n)]

def run_multiprocess(chunks, function, poolsize=2):
    with Pool(poolsize) as p:
        return_data=p.map(function, chunks)
    return return_data

In [121]:
### Helper Functions 


# feat1 fsf file creation 
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict, design_file, newkey, run):
    
    with open(design_file,'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        #print(output_dir)
        
        
        task_name=trial_id.replace(sub_id+'_',"")
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_beta1.fsf"%(sub_id, ses_id, task_name))
        out_param = data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][newkey]["FUNCRUN"]
        confound_param = data_dict[sub_id][newkey]["CONFOUND"]
        trial_param = data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]
        #print(design_fileout)
        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][newkey]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        #print(tempfsf)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()
        
        
def create_fsf(input_dir, deriv_dir, ses_id, design_file, ev_path):
    
    ses_id=ses_id
    preproc_folder = deriv_dir
    
    data_dict={}
    
    
    # start loop -- looping through subjects
    subject_list = glob.glob(os.path.join(preproc_folder, 'sub-*/%s'%ses_id))
    for sub_path in subject_list:
        # set variables from file path
        sub_id=sub_path.split("/")[-2]
        # add subject id to dictionary
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
            
            
            
        # get functional task files 
        # this section may be unique to an individual study,
        # make sure the extraction applies correctly. 
        
        # get list of nifti functional files of current subject
        # we use the `_preproc_bold_brain.nii.gz task files, these are preprocessed niftis
        functional_tasks = glob.glob(os.path.join(sub_path, 'func/*preproc_bold_brain.nii.gz'))
                
        
        for functional in functional_tasks: 
            # set specific variables from filename
            task=functional.split("/")[-1].split("_")[2].split("-")[1]
            if 'resting' in task:
                run=""
                pass
            else:
                run=functional.split("/")[-1].split("_")[3]

                
                #print(task,run)

                analysis_folder=os.path.join(deriv_dir, '%s/%s'%(sub_id,ses_id), "analysis")
                #print(analysis_folder)
                output_dir = os.path.join(analysis_folder, 'beta/task-%s_%s'%(task,run))
                #print('[INFO] output directory: ', output_dir)
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                # set confound file
                #sub-001_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_confound.txt

                if 'resting' in task:
                    confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt"%(sub_id,ses_id, task))
                else:
                    confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt"%(sub_id,ses_id, task, run))

                #print(confound_file)
                #print(run)
                if 'run' in run:
                    newkey="%s_%s"%(task, run)
                else:
                    newkey="%s"%task
                #print(newkey)
                data_dict[sub_id][newkey] = {
                    "TRIALS" : { },
                    "CONFOUND" : confound_file,
                    "FUNCRUN" : functional
                  }           

                # set motion parameters
                # sub-001_ses-1_task-training_run-1_moco0.txt | sub-001_ses-1_task-resting_moco5.txt
                for i in range(6):
                    if 'run' in run:
                        motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_%s_moco%s.txt'%(sub_id,ses_id, task,run,i))
                    else:
                        motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_moco%s.txt'%(sub_id,ses_id, task,i))
                    #print(motcor)
                    data_dict[sub_id][newkey]['MOCO%i'%i] = motcor

                #print(data_dict)

                # setup evs(onsets)
                trial_evs = sorted(glob.glob(os.path.join(ev_path, '%s_*%s*.txt'%(sub_id, run))))
                #print(trial_evs[0])

                if not trial_evs:
                    pass
                else:
                    for trial_file in trial_evs:
                        _id = sub_id.split("-")[1]
                        _id = _id[1:]
                        trial_id = trial_file.split("/")[-1].split(".")[0]
                        #print(trial_id)
                        nuis_file = os.path.join(ev_path, '%s.txt'%trial_id.replace('trial', 'nuis'))
                        #print(nuis_file)
                        
                        trial_ext=trial_id.replace(sub_id+'_',"")
                        fileout = os.path.join(output_dir, "%s_%s_%s"%(sub_id, ses_id,trial_ext))
                        #print(fileout)
                        
                        # fill dictionary
                        data_dict[sub_id][newkey]["TRIALS"]["TRIAL%s"%trial_id] = {"TRIAL" : trial_file, "NUIS": nuis_file, "OUTPUT" : fileout}
                        
                        # write out file
                        make_file(sub_id, ses_id, trial_id,output_dir, task, data_dict,design_file, newkey, run)

                
    return(data_dict) 
        

## Setup Feat files

In [123]:
# make feat1 fsf files
print('[INFO] making fsf design files...')
#data_dict=create_fsf(input_dir, deriv_dir,ses_id,design_file,ev_path)
print('[INFO] process complete.')


[INFO] making fsf design files...
[INFO] process complete.


Lets view a few of files created

In [28]:
# look at created files
output_path='/projects/niblab/experiments/bbx/data/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files'
listdir(output_path)[:5]
        

['sub-001_ses-1_task-H2O-run-4_nuis1_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis2_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis3_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis4_beta1.fsf',
 'sub-001_ses-1_task-H2O-run-4_nuis5_beta1.fsf']

In [29]:
beta1_files_path='/projects/niblab/experiments/bbx/data/preprocessed/sub-*/ses-1/analysis/beta/task-training_run-*/design_files/*.fsf'
print('[INFO] %f fsf files available.'%len(glob.glob(beta1_files_path)))


[INFO] 64784.000000 fsf files available.


In [6]:
# view a few of the files
glob.glob(beta1_files_path)[:5]

['/projects/niblab/experiments/bbx/data/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis1_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis2_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis3_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis4_beta1.fsf',
 '/projects/niblab/experiments/bbx/data/preprocessed/sub-001/ses-1/analysis/beta/task-training_run-4/design_files/sub-001_ses-1_task-H2O-run-4_nuis5_beta1.fsf']

## Submit slurm jobs

Tasks Available

In [124]:
len(subject_ids)

129

In [30]:
tasks=[x.split("/")[-1].split("_")[2].split("-")[1] for x in glob.glob(output_path+"/*.fsf")]
set(tasks)


{'H2O', 'H2Ocue', 'SSB', 'SSBcue', 'USB', 'USBcue', 'rinse'}

In [127]:
def by_file_slurm(subject,ev_task):
    fsf_path='/projects/niblab/experiments/bbx/data/preprocessed/{}/ses-1/analysis/beta/task-training_run-*/design_files/*{}*.fsf'.format(subject, ev_task)
    fsfs=glob.glob(fsf_path)
    for fsf in fsfs:
        slurm_cmd = "sbatch /projects/niblab/experiments/bbx/data/code/beta_by_file.job {}".format(fsf)
        #print('[INFO] submitted: \n', slurm_cmd)
        os.system(slurm_cmd)
    


In [128]:
# run all jobs
print('[INFO] setting up batch jobs.')
ev_task='USB'
for subject in subject_ids[20:60]:    
    by_file_slurm(subject, ev_task)
    
print('[INFO] submitted batch jobs.')

    

[INFO] setting up batch jobs.
[INFO] submitted batch jobs.


In [18]:
#!squeue -u nbytes

## Quality Check

**Setup Data Dictionary**

In [100]:
bad_subjects=[]
for folder in subject_folders:
    subject_id=folder.split("/")[-1]
    tasks=glob.glob(os.path.join(folder,'ses-1/analysis/beta/*'))
    #print('[INFO] tasks: ', tasks)
    
    if not tasks:
        bad_subjects.append(subject_id)
    else:
        if subject_id not in data_dict:
            data_dict[subject_id]={}
        for task_folder in tasks:
            #print(task_folder)
            task=task_folder.split("/")[-1].split('.')[0]
            #print('[INFO] task: ', task)
            if task not in data_dict[subject_id]:
                data_dict[subject_id][task]={}
            pes=glob.glob(os.path.join(task_folder, '*.feat/stats/pe1.nii.gz'))
            data_dict[subject_id][task]=pes
       
    
    

In [101]:
#data_dict['sub-001']['task-training_run-1']

## Concatenate PEs

In [116]:
keyword="USBcue"
bad_subjects=[]

In [117]:

def merge_files(subject_list, data_dict=data_dict,keyword=keyword):

    for subject_id in subject_list[:2]:
        for task in data_dict[subject_id]:
            #print('[INFO] task: ', task)
            pes=[pe for pe in data_dict[subject_id][task] if keyword in pe]
            if "-" in keyword:
                keyword=keyword.strip('-')
                
            #print("[INFO] PEs: ",pes)
            filename="%s_%s_%s"%(subject_id, task, keyword)
            outfile = "/projects/niblab/experiments/bbx/data/betaseries/niftis/%s/%s"%(keyword, filename)
            #print('[INFO] outfile: \n',outfile)
            if not os.path.exists("/projects/niblab/experiments/bbx/data/betaseries/niftis/%s"%keyword):
                 os.makedirs("/projects/niblab/experiments/bbx/data/betaseries/niftis/%s"%keyword)
                    
            if not pes:
                bad_subjects.append(subject_id)
            else:
                pe_str = " ".join(pes)
                fslmerge_cmd="/projects/niblab/modules/software/fsl/5.0.10/bin/fslmerge -t %s %s"%(outfile, pe_str)
                #print("[INFO] running fsl merge command...")
                #print(fslmerge_cmd, "\n")
                os.system(fslmerge_cmd)


            

In [118]:
chunksize=10
print("[INFO] chunksize: {}".format(chunksize))
chunk_list=chunks(subject_ids, chunksize)

#print(chunk_list)
print("[INFO] running fslmerge ....\nfslmerge -t [OUTPUT FILENAME][PE FILES]")

with Pool(8) as p:
    p.map(merge_files, chunk_list)
print("[INFO] process complete.")
#print('[INFO] find images here: ',concat_path )

[INFO] chunksize: 10
[INFO] running fslmerge ....
fslmerge -t [OUTPUT FILENAME][PE FILES]
[INFO] process complete.


In [107]:
def tranform_niftis(niftis, keyword=keyword):
    reference_nifti='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.nii.gz'
    reference_mat='/projects/niblab/parcellations/chocolate_decoding_rois/mni2ace.mat'
    for nii in niftis:

        # setup and run flirt
        nii=nii.replace('.nii.gz', '')
        out=nii#+'_3mm'
        flirt_cmd="flirt -in {} -ref {} -init {} -applyxfm -out {}".format(nii, reference_nifti, reference_mat, out)
        #print('[INFO] flirt command: \n{}'.format(flirt_cmd))
        os.system(flirt_cmd)

        fslmaths_cmd='fslmaths {} -thr 0.9 {}'.format(out,out)
        #print('[INFO] fslmaths command: \n{}'.format(fslmaths_cmd))
        os.system(fslmaths_cmd)

In [108]:

print('[INFO] transform functionals to match the mask.')
chunksize=10
# grab concatenated (fslmerge) data
keyword="SSBcue"
fslmerged_files=glob.glob(os.path.join('/projects/niblab/experiments/bbx/data/betaseries/niftis/%s/*.nii.gz'%(keyword)))
#fslmerged_files[:4]
print("[INFO] chunksize: {}".format(chunksize))
chunk_list=chunks(fslmerged_files, chunksize)
with Pool(8) as p:
    p.map(tranform_niftis, chunk_list)
print('[INFO] transformation process complete.')


[INFO] transform functionals to match the mask.
[INFO] chunksize: 10
[INFO] transformation process complete.


In [93]:
!sbatch /projects/niblab/experiments/bbx/data/betaseries/timeseries_roi_pull.job SSB


Submitted batch job 3274019


In [95]:
!squeue -j 3274019

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           3274019     batch timeseri   nbytes  R       0:43      1 largemem-0-0
